<a href="https://colab.research.google.com/github/marianalreis/kaggle_project/blob/main/kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de Sucesso de Startups
### Mariana Lacerda Reis - T16
Esse projeto tem como objetivo prever se uma startup terá sucesso ou fracasso usando dados como idade, setor, investimentos e localização, aplicando modelos de aprendizado de máquina para identificar padrões.

**1º passo:** Importar todas as bibliotecas necessárias

In [ ]:
!pip install scikit-learn pandas numpy

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

print(train.shape, test.shape)
train.head()

(646, 33) (277, 32)


,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,labels
0,719,10.42,13.09,8.98,12.72,4,3,4087500,3,1,...,0,0,1,1,0,0,0,0,1.0,0
1,429,3.79,3.79,NaN,NaN,21,1,45000000,0,0,...,0,0,0,0,0,1,0,0,1.0,1
2,178,0.71,2.28,1.95,2.28,5,2,5200000,2,1,...,0,1,1,0,1,0,0,0,1.0,0
3,197,3.00,5.00,9.62,10.39,16,2,14500000,2,0,...,0,0,0,1,0,1,0,0,2.0,1
4,444,0.66,5.88,6.21,8.61,29,5,70000000,4,1,...,0,0,0,0,1,1,1,1,2.8,1


## Data exploration
Nessa etapa, vou estudar as informações que a tabela oferece:

Começando pelo arquivo **train**

In [ ]:
train.head()

,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,labels
0,719,10.42,13.09,8.98,12.72,4,3,4087500,3,1,...,0,0,1,1,0,0,0,0,1.0,0
1,429,3.79,3.79,NaN,NaN,21,1,45000000,0,0,...,0,0,0,0,0,1,0,0,1.0,1
2,178,0.71,2.28,1.95,2.28,5,2,5200000,2,1,...,0,1,1,0,1,0,0,0,1.0,0
3,197,3.00,5.00,9.62,10.39,16,2,14500000,2,0,...,0,0,0,1,0,1,0,0,2.0,1
4,444,0.66,5.88,6.21,8.61,29,5,70000000,4,1,...,0,0,0,0,1,1,1,1,2.8,1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        646 non-null    int64  
 1   age_first_funding_year    611 non-null    float64
 2   age_last_funding_year     637 non-null    float64
 3   age_first_milestone_year  508 non-null    float64
 4   age_last_milestone_year   535 non-null    float64
 5   relationships             646 non-null    int64  
 6   funding_rounds            646 non-null    int64  
 7   funding_total_usd         646 non-null    int64  
 8   milestones                646 non-null    int64  
 9   is_CA                     646 non-null    int64  
 10  is_NY                     646 non-null    int64  
 11  is_MA                     646 non-null    int64  
 12  is_TX                     646 non-null    int64  
 13  is_otherstate             646 non-null    int64  
 14  category_c

In [ ]:
train.columns

Index(['id', 'age_first_funding_year', 'age_last_funding_year',
       'age_first_milestone_year', 'age_last_milestone_year', 'relationships',
       'funding_rounds', 'funding_total_usd', 'milestones', 'is_CA', 'is_NY',
       'is_MA', 'is_TX', 'is_otherstate', 'category_code', 'is_software',
       'is_web', 'is_mobile', 'is_enterprise', 'is_advertising',
       'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting',
       'is_othercategory', 'has_VC', 'has_angel', 'has_roundA', 'has_roundB',
       'has_roundC', 'has_roundD', 'avg_participants', 'labels'],
      dtype='object')

In [ ]:
train.describe()

,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,labels
count,646.000000,611.000000,637.000000,508.000000,535.000000,646.000000,646.000000,6.460000e+02,646.000000,646.000000,...,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000
mean,461.577399,2.341718,4.037724,3.352657,4.944729,7.948916,2.351393,2.949633e+07,1.913313,0.546440,...,0.003096,0.304954,0.329721,0.260062,0.515480,0.419505,0.235294,0.091331,2.848655,0.647059
std,264.859464,2.468275,2.950923,2.866952,3.213319,7.397602,1.357856,2.261999e+08,1.337095,0.498224,...,0.055598,0.460745,0.470476,0.439008,0.500148,0.493860,0.424511,0.288303,1.894050,0.478255
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.100000e+04,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,233.250000,0.680000,1.870000,1.185000,2.540000,3.000000,1.000000,3.000000e+06,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,0.000000
50%,459.500000,1.650000,3.610000,2.785000,4.620000,6.000000,2.000000,1.020000e+07,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.333300,1.000000
75%,692.500000,3.600000,5.590000,4.935000,6.880000,10.000000,3.000000,2.587500e+07,3.000000,1.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,1.000000
max,923.000000,21.900000,21.900000,24.680000,24.680000,63.000000,8.000000,5.700000e+09,6.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16.000000,1.000000


In [ ]:
train.shape

(646, 33)

In [ ]:
train.isnull().sum()

,0
id,0
age_first_funding_year,35
age_last_funding_year,9
age_first_milestone_year,138
age_last_milestone_year,111
relationships,0
funding_rounds,0
funding_total_usd,0
milestones,0
is_CA,0


**Labels** é a coluna mais importante pois é a que diz a resposta da nossa pergunta, então vou ver como são exibidas e a quantodade de respostas que tem 1 e 0 nela.

In [ ]:
train[["id", "labels"]].head(15)

,id,labels
0,719,0
1,429,1
2,178,0
3,197,1
4,444,1
5,67,0
6,505,0
7,410,1
8,284,0
9,252,1


In [ ]:
train["labels"].value_counts()

,count
labels,
1,418
0,228


Agora, explorei os dados do **test**

In [ ]:
test.head()

,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_biotech,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants
0,70,1.33,3.02,3.02,3.34,9,3,11489687,2,1,...,0,0,0,0,1,1,1,0,0,3.3333
1,23,5.49,5.49,0.00,0.00,1,1,3170000,1,1,...,0,0,0,0,0,1,0,0,0,2.0000
2,389,5.84,8.22,NaN,NaN,4,3,47218498,0,1,...,0,0,1,1,0,0,0,1,1,3.3333
3,872,6.30,6.30,6.00,6.00,9,1,15000000,1,1,...,1,0,0,0,0,0,0,0,0,3.0000
4,920,7.25,9.23,6.00,6.00,1,3,52000000,1,0,...,0,0,1,1,0,0,1,0,0,2.6667


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        277 non-null    int64  
 1   age_first_funding_year    266 non-null    float64
 2   age_last_funding_year     273 non-null    float64
 3   age_first_milestone_year  217 non-null    float64
 4   age_last_milestone_year   224 non-null    float64
 5   relationships             277 non-null    int64  
 6   funding_rounds            277 non-null    int64  
 7   funding_total_usd         277 non-null    int64  
 8   milestones                277 non-null    int64  
 9   is_CA                     277 non-null    int64  
 10  is_NY                     277 non-null    int64  
 11  is_MA                     277 non-null    int64  
 12  is_TX                     277 non-null    int64  
 13  is_otherstate             277 non-null    int64  
 14  category_c

In [ ]:
test.columns

Index(['id', 'age_first_funding_year', 'age_last_funding_year',
       'age_first_milestone_year', 'age_last_milestone_year', 'relationships',
       'funding_rounds', 'funding_total_usd', 'milestones', 'is_CA', 'is_NY',
       'is_MA', 'is_TX', 'is_otherstate', 'category_code', 'is_software',
       'is_web', 'is_mobile', 'is_enterprise', 'is_advertising',
       'is_gamesvideo', 'is_ecommerce', 'is_biotech', 'is_consulting',
       'is_othercategory', 'has_VC', 'has_angel', 'has_roundA', 'has_roundB',
       'has_roundC', 'has_roundD', 'avg_participants'],
      dtype='object')

In [ ]:
test.describe()

,id,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,is_CA,...,is_biotech,is_consulting,is_othercategory,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants
count,277.000000,266.000000,273.000000,217.000000,224.000000,277.000000,277.000000,2.770000e+02,277.000000,277.000000,...,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000
mean,462.985560,2.497857,3.924286,3.312535,4.663348,7.155235,2.216606,1.591263e+07,1.675090,0.483755,...,0.032491,0.003610,0.364621,0.317690,0.241877,0.490975,0.328520,0.227437,0.119134,2.815103
std,271.068618,2.446842,2.825445,2.553868,2.847457,6.929857,1.463324,2.068785e+07,1.275122,0.500641,...,0.177621,0.060084,0.482195,0.466421,0.428995,0.500823,0.470525,0.419936,0.324532,1.831605
min,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000e+04,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,215.000000,0.722500,1.760000,1.250000,2.485000,3.000000,1.000000,2.238055e+06,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,464.000000,1.580000,3.510000,2.970000,4.165000,5.000000,2.000000,9.000000e+06,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.500000
75%,692.000000,4.025000,5.560000,5.000000,6.722500,9.000000,3.000000,2.225000e+07,2.000000,1.000000,...,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,3.500000
max,922.000000,16.420000,16.420000,13.010000,13.010000,57.000000,10.000000,1.622641e+08,8.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.000000


In [ ]:
test.shape

(277, 32)

In [ ]:
test.isnull().sum()

,0
id,0
age_first_funding_year,11
age_last_funding_year,4
age_first_milestone_year,60
age_last_milestone_year,53
relationships,0
funding_rounds,0
funding_total_usd,0
milestones,0
is_CA,0


## Preprocessing
Nessa etapa, vou preparar meus dados para o modelo.Como está sendo pedido na instrução do exercício, o que queremos é um modelo supervisionado de classificação. Pois vamos prever se a start up irá dar certo ou não, como uma classificação binária (1 e 0).

Então, para iniciar o processamento, precisamos separar x, que é a entrada, de y que é o que vai voltar de dados, como uma resposta.

In [ ]:
TARGET_COL = "labels"
ID_COL = "id"

BIN_DUMMIES = [
    'is_CA','is_NY','is_MA','is_TX','is_otherstate',
    'is_software','is_web','is_mobile','is_enterprise',
    'is_advertising','is_gamesvideo','is_ecommerce',
    'is_biotech','is_consulting','is_othercategory',
    'has_VC','has_angel','has_roundA','has_roundB','has_roundC','has_roundD'
]

NUM_CONTINUOUS = [
    'age_first_funding_year','age_last_funding_year',
    'age_first_milestone_year','age_last_milestone_year',
    'relationships','funding_rounds','funding_total_usd',
    'milestones','avg_participants'
]

CAT_RAW = ['category_code']

def load_data(train_path="train.csv", test_path="test.csv", sample_path="sample_submission.csv"):
    train = pd.read_csv(train_path)
    test  = pd.read_csv(test_path)
    sample= pd.read_csv(sample_path)
    return train, test, sample

def split_xy(train):
    X = train.drop(columns=[TARGET_COL, ID_COL])
    y = train[TARGET_COL]
    return X, y

def build_preprocess():
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler())
    ])
    binary_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent"))
    ])
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])
    preprocess = ColumnTransformer(transformers=[
        ("num", numeric_transformer, NUM_CONTINUOUS),
        ("bin", binary_transformer, BIN_DUMMIES),
        ("cat", categorical_transformer, CAT_RAW)
    ])
    return preprocess

def get_test_features(test_df):
    return test_df.drop(columns=[ID_COL], errors="ignore")

## Criar modelos

### Modelo 1- Regressão Logística

In [ ]:
SEED = 42

def main():
    train, test, sample = load_data()
    X, y = split_xy(train)
    preprocess = build_preprocess()

    pipe = Pipeline(steps=[
        ("prep", preprocess),
        ("model", LogisticRegression(max_iter=1000, class_weight="balanced", random_state=SEED))
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    auc = cross_val_score(pipe, X, y, scoring="roc_auc", cv=cv, n_jobs=-1).mean()
    print(f"[LogReg] CV ROC AUC: {auc:.4f}")

    # Treino final e submissão
    pipe.fit(X, y)
    test_features = get_test_features(test)
    preds = pipe.predict_proba(test_features)[:, 1]

    sub = sample.copy()
    sub.iloc[:, 1] = pd.Series(preds, index=test[ID_COL].values).reindex(sub.iloc[:,0]).values
    sub.to_csv("submission_logreg.csv", index=False)
    print("Gerado: submission_logreg.csv")

if __name__ == "__main__":
    main()

[LogReg] CV ROC AUC: 0.7842
Gerado: submission_logreg.csv


/tmp/ipython-input-1799488832.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.63299587 0.15054442 0.59133599 0.70132854 0.52713938 0.78698451
 0.23735487 0.31560759 0.581616   0.5885041  0.63639397 0.96803948
 0.6967126  0.27003279 0.12183601 0.58130682 0.87472759 0.06346842
 0.48609324 0.36514994 0.64969393 0.10667252 0.73411247 0.50040933
 0.18128356 0.76278269 0.95114408 0.4883656  0.25587935 0.21213577
 0.3563684  0.36269545 0.20332216 0.75863044 0.87271826 0.16830287
 0.20735919 0.96148339 0.12476702 0.86659463 0.76908763 0.93560437
 0.01901348 0.23334196 0.17974055 0.31584218 0.86343633 0.33496434
 0.51096088 0.38735551 0.50662912 0.35851145 0.66622781 0.50188892
 0.9282779  0.69457529 0.20934977 0.18185631 0.32855176 0.07651771
 0.14019087 0.70273043 0.14743183 0.11359015 0.33202581 0.24600993
 0.27432389 0.77728372 0.11425105 0.59835409 0.70823965 0.56141856
 0.40687156 0.61826831 0.89780962 0.66229

### Modelo 2 - Random Forest

In [ ]:
def main():
    train, test, sample = load_data()
    X, y = split_xy(train)
    preprocess = build_preprocess()

    pipe = Pipeline(steps=[
        ("prep", preprocess),
        ("model", RandomForestClassifier(n_estimators=400, random_state=SEED, n_jobs=-1))
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    auc = cross_val_score(pipe, X, y, scoring="roc_auc", cv=cv, n_jobs=-1).mean()
    print(f"[RandomForest] CV ROC AUC: {auc:.4f}")

    pipe.fit(X, y)
    test_features = get_test_features(test)
    preds = pipe.predict_proba(test_features)[:, 1]

    sub = sample.copy()
    sub.iloc[:, 1] = pd.Series(preds, index=test[ID_COL].values).reindex(sub.iloc[:,0]).values
    sub.to_csv("submission_rf.csv", index=False)
    print("Gerado: submission_rf.csv")

if __name__ == "__main__":
    main()

[RandomForest] CV ROC AUC: 0.8057
Gerado: submission_rf.csv


/tmp/ipython-input-157643554.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.965  0.2525 0.7875 0.8175 0.5075 0.64   0.1925 0.46   0.635  0.7475
 0.88   0.915  0.7225 0.4475 0.15   0.7825 0.8475 0.0575 0.72   0.1125
 0.8625 0.2475 0.69   0.6125 0.1275 0.825  0.9275 0.75   0.1625 0.3
 0.655  0.1975 0.3425 0.7725 0.8875 0.56   0.34   0.7925 0.4475 0.805
 0.91   0.87   0.0175 0.3    0.34   0.22   0.7625 0.3225 0.805  0.6275
 0.85   0.6525 0.8875 0.7725 0.83   0.4975 0.3275 0.45   0.26   0.1175
 0.05   0.75   0.28   0.125  0.3175 0.645  0.6    0.935  0.1975 0.775
 0.9175 0.795  0.69   0.655  0.895  0.505  0.9175 0.6025 0.6475 0.5475
 0.7725 0.84   0.8325 0.275  0.49   0.845  0.29   0.69   0.635  0.6925
 0.935  0.9425 0.7975 0.83   0.91   0.7825 0.66   0.88   0.77   0.8575
 0.89   0.665  0.6875 0.435  0.6075 0.7425 0.8125 0.9425 0.4425 0.7625
 0.6975 0.76   0.155  0.6625 0.8125 0.7775 0.755  0.7625 0.7475 0.6775

### Modelo 3 - G boosting

In [ ]:
def main():
    train, test, sample = load_data()
    X, y = split_xy(train)
    preprocess = build_preprocess()

    pipe = Pipeline(steps=[
        ("prep", preprocess),
        ("model", HistGradientBoostingClassifier(random_state=SEED))
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    auc = cross_val_score(pipe, X, y, scoring="roc_auc", cv=cv, n_jobs=-1).mean()
    print(f"[HGB] CV ROC AUC: {auc:.4f}")

    pipe.fit(X, y)
    test_features = get_test_features(test)
    preds = pipe.predict_proba(test_features)[:, 1]

    sub = sample.copy()
    sub.iloc[:, 1] = pd.Series(preds, index=test[ID_COL].values).reindex(sub.iloc[:,0]).values
    sub.to_csv("submission_hgb.csv", index=False)
    print("Gerado: submission_hgb.csv")

if __name__ == "__main__":
    main()

[HGB] CV ROC AUC: 0.7842
Gerado: submission_hgb.csv


/tmp/ipython-input-3068150987.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[9.95051307e-01 5.72701732e-02 9.64414718e-01 9.37108142e-01
 1.23114969e-01 7.43942677e-01 6.43707815e-02 5.25553904e-01
 7.28787954e-01 9.55562611e-01 9.02029595e-01 9.85490337e-01
 6.98212925e-01 2.49537747e-01 6.58705291e-02 8.93668667e-01
 9.89182529e-01 1.55413138e-02 9.70911262e-01 2.99265313e-03
 9.64368072e-01 3.74771509e-02 8.98928393e-01 3.64608742e-01
 7.45769264e-03 9.81588719e-01 9.75027344e-01 6.29024383e-01
 2.59373842e-02 3.69430887e-02 7.48930376e-01 6.73357489e-02
 3.94374685e-01 9.75933589e-01 9.97376045e-01 9.21802674e-01
 2.16244946e-02 9.74316612e-01 1.11346180e-01 9.56865985e-01
 9.99179654e-01 9.99195924e-01 1.58254867e-03 1.42344611e-01
 2.80964607e-01 3.68087266e-02 8.00171886e-01 4.12523028e-02
 9.73237342e-01 8.14210997e-01 9.23209445e-01 9.13582485e-01
 9.49318322e-01 6.96576528e-01 9.97894338e-01 3.9501

### Comparação dos modelos

In [ ]:
SEED = 42

X, y = split_xy(train)
preprocess = build_preprocess()

logreg_pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced", random_state=SEED))
])

# Modelo 2 — Random Forest
rf_pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", RandomForestClassifier(n_estimators=400, random_state=SEED, n_jobs=-1))
])

# Modelo 3 — HistGradientBoosting
hgb_pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("model", HistGradientBoostingClassifier(random_state=SEED))
])


# Validação cruzada e cálculo da acurácia
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
acc_logreg = cross_val_score(logreg_pipe, X, y, scoring="accuracy", cv=cv, n_jobs=-1).mean()
acc_rf     = cross_val_score(rf_pipe,     X, y, scoring="accuracy", cv=cv, n_jobs=-1).mean()
acc_hgb    = cross_val_score(hgb_pipe,    X, y, scoring="accuracy", cv=cv, n_jobs=-1).mean()

print(f"Acurácia (CV) — LogReg: {acc_logreg:.4f} | RF: {acc_rf:.4f} | HGB: {acc_hgb:.4f}")


Acurácia (CV) — LogReg: 0.7198 | RF: 0.7879 | HGB: 0.7569


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
acc_logreg = cross_val_score(logreg_pipe, X, y, scoring="accuracy", cv=cv, n_jobs=-1).mean()
acc_rf     = cross_val_score(rf_pipe,     X, y, scoring="accuracy", cv=cv, n_jobs=-1).mean()
acc_hgb    = cross_val_score(hgb_pipe,    X, y, scoring="accuracy", cv=cv, n_jobs=-1).mean()

print(f"Acurácia (CV) — LogReg: {acc_logreg:.4f} | RF: {acc_rf:.4f} | HGB: {acc_hgb:.4f}")

Acurácia (CV) — LogReg: 0.7198 | RF: 0.7879 | HGB: 0.7569


In [ ]:
from sklearn.metrics import make_scorer, f1_score, roc_auc_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

results = []

for name, pipe in [("LogReg", logreg_pipe), ("RandomForest", rf_pipe), ("HGB", hgb_pipe)]:
    acc  = cross_val_score(pipe, X, y, scoring="accuracy", cv=cv, n_jobs=-1).mean()
    f1   = cross_val_score(pipe, X, y, scoring="f1", cv=cv, n_jobs=-1).mean()
    auc  = cross_val_score(pipe, X, y, scoring="roc_auc", cv=cv, n_jobs=-1).mean()
    results.append({"Modelo": name, "Acurácia": acc, "F1": f1, "ROC AUC": auc})

results_df = pd.DataFrame(results).sort_values(by="ROC AUC", ascending=False)
print(results_df)

         Modelo  Acurácia        F1   ROC AUC
1  RandomForest  0.787859  0.846466  0.805654
0        LogReg  0.719809  0.769586  0.784237
2           HGB  0.756899  0.819282  0.784168


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "model__n_estimators": [200, 400, 800],
    "model__max_depth": [None, 5, 10, 20],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ["sqrt", "log2"]
}

In [ ]:
# ==== TUNING DO RANDOM FOREST ====
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

# 1) Pipeline base com o mesmo preprocess
rf_base = Pipeline([
    ("prep", preprocess),
    ("model", RandomForestClassifier(random_state=SEED, n_jobs=-1))
])

# 2) Espaço de busca
param_grid = {
    "model__n_estimators": [300, 500, 800, 1200],
    "model__max_depth": [None, 8, 12, 20, 30],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ["sqrt", "log2", 0.5],
    "model__class_weight": [None, "balanced"]
}

# 3) Validação e busca
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
search = RandomizedSearchCV(
    rf_base,
    param_distributions=param_grid,
    n_iter=40,              # pode aumentar para explorar mais
    scoring="accuracy",     # troque para "roc_auc" se a métrica oficial for AUC
    cv=cv,
    n_jobs=-1,
    random_state=SEED,
    verbose=1
)

search.fit(X, y)
print("Melhor ACC (CV):", round(search.best_score_, 4))
print("Melhores params:", search.best_params_)

best_rf = search.best_estimator_


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Melhor ACC (CV): 0.7957
Melhores params: {'model__n_estimators': 500, 'model__min_samples_split': 5, 'model__min_samples_leaf': 1, 'model__max_features': 0.5, 'model__max_depth': None, 'model__class_weight': 'balanced'}


In [ ]:
# Treinar modelo escolhido (ex.: Random Forest)
rf_pipe.fit(X, y)

# Gerar previsões como 0/1
test_features = get_test_features(test)
preds = rf_pipe.predict(test_features)

# Montar submissão no formato do sample
sub = sample.copy()
sub.iloc[:, 1] = pd.Series(preds, index=test[ID_COL].values).reindex(sub.iloc[:,0]).values

# Salvar o arquivo final
sub.to_csv("submission.csv", index=False)


In [ ]:
best_rf = search.best_estimator_
best_rf.fit(X, y)

# features do teste
test_features = get_test_features(test)

# >>> PREDIÇÃO BINÁRIA 0/1
preds = best_rf.predict(test_features).astype(int)

# montar submissão exatamente como o sample
sub = sample.copy()
sub.iloc[:, 1] = pd.Series(preds, index=test[ID_COL]).reindex(sub.iloc[:, 0]).values

sub.to_csv("submission.csv", index=False)
print("Arquivo pronto: submission.csv")

Arquivo pronto: submission.csv


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score
import numpy as np

# 1) OOF probs com o melhor RF
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
oof_proba = cross_val_predict(best_rf, X, y, cv=cv, method="predict_proba", n_jobs=-1)[:,1]

# 2) varre thresholds e pega o que maximiza ACC
ths = np.linspace(0.2, 0.8, 241)  # de 0.2 a 0.8 de 0.002 em 0.002
accs = [accuracy_score(y, (oof_proba >= t).astype(int)) for t in ths]
best_t = float(ths[int(np.argmax(accs))])
print(f"Melhor limiar CV: {best_t:.3f} | ACC CV: {max(accs):.4f}")

# 3) re-treina no full e aplica no TEST usando esse limiar
best_rf.fit(X, y)
test_proba = best_rf.predict_proba(get_test_features(test))[:,1]
preds = (test_proba >= best_t).astype(int)

sub = sample.copy()
sub.iloc[:,1] = pd.Series(preds, index=test[ID_COL]).reindex(sub.iloc[:,0]).values
sub.to_csv("submission_threshold.csv", index=False)
print("Gerado: submission_threshold.csv")

Melhor limiar CV: 0.488 | ACC CV: 0.8019
Gerado: submission_threshold.csv


In [ ]:
# treina os dois no full
rf_best = best_rf.fit(X, y)
hgb_best = hgb_pipe.fit(X, y)

# OOF p/ achar limiar do ensemble
rf_oof = cross_val_predict(rf_best, X, y, cv=cv, method="predict_proba", n_jobs=-1)[:,1]
hgb_oof = cross_val_predict(hgb_best, X, y, cv=cv, method="predict_proba", n_jobs=-1)[:,1]
ens_oof = 0.5*rf_oof + 0.5*hgb_oof

ths = np.linspace(0.2, 0.8, 241)
accs = [accuracy_score(y, (ens_oof >= t).astype(int)) for t in ths]
best_t_ens = float(ths[int(np.argmax(accs))])
print(f"Ensemble: melhor limiar {best_t_ens:.3f} | ACC CV {max(accs):.4f}")

# aplica no TEST
rf_p = rf_best.predict_proba(get_test_features(test))[:,1]
hgb_p = hgb_best.predict_proba(get_test_features(test))[:,1]
ens_p = 0.5*rf_p + 0.5*hgb_p
preds = (ens_p >= best_t_ens).astype(int)

sub = sample.copy()
sub.iloc[:,1] = pd.Series(preds, index=test[ID_COL]).reindex(sub.iloc[:,0]).values
sub.to_csv("submission_ensemble.csv", index=False)
print("Gerado: submission_ensemble.csv")


Ensemble: melhor limiar 0.285 | ACC CV 0.7833
Gerado: submission_ensemble.csv


In [ ]:
def engineer_features(df):
    out = df.copy()
    r = out['funding_rounds'].replace(0, 1)  # evita div/0
    out['funding_per_round'] = out['funding_total_usd'] / r
    out['milestones_per_round'] = out['milestones'] / r
    out['rels_per_round'] = out['relationships'] / r
    return out

# aplique logo após ler os CSVs, antes de split/preprocess
train = engineer_features(train)
test  = engineer_features(test)

# e adicione essas 3 colunas em NUM_CONTINUOUS:
# NUM_CONTINUOUS += ['funding_per_round','milestones_per_round','rels_per_round']


In [ ]:
p_test = best_rf.predict_proba(get_test_features(test))[:, 1]

# aplica o limiar CV ótimo
thr = 0.488
preds = (p_test >= thr).astype(int)

# monta submissão exatamente como o sample
sub = sample.copy()
sub.iloc[:, 1] = pd.Series(preds, index=test[ID_COL]).reindex(sub.iloc[:, 0]).values
sub.to_csv("submission_threshold.csv", index=False)
print("Gerado: submission_threshold.csv")

Gerado: submission_threshold.csv


In [ ]:
def engineer_features(df):
    out = df.copy()
    r = out['funding_rounds'].replace(0, 1)
    out['funding_per_round']   = out['funding_total_usd'] / r
    out['milestones_per_round']= out['milestones'] / r
    out['rels_per_round']      = out['relationships'] / r
    return out

# aplique ANTES de split/preprocess
train = engineer_features(train)
test  = engineer_features(test)

# adicione no NUM_CONTINUOUS (uma vez só)
for c in ['funding_per_round','milestones_per_round','rels_per_round']:
    if c not in NUM_CONTINUOUS:
        NUM_CONTINUOUS.append(c)


In [ ]:
from google.colab import files
files.download("submission_threshold.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>